#### Aprendizaje supervisado: cada observación tiene una respuesta asociada que guía el aprendizaje

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn import tree


df  = pd.read_csv("data/titanic.csv")

df.info()

#### Contexto y problema
Definir un problema en Machine Learning o Aprendizaje Automático requiere de una serie de pasos hasta llegar a una conclusión que defina un problema a resolver. En este caso no se trata de un problema real o necesidad que una empresa debería resolver para mejorar las decisones que elige ni tampoco anticipar como se encotrará en el futuro, sino más bien se trata de un ejemplo sobre como implementar y traducir los distintos componentes de un problema didáctico al Machine Learning. Entonces, introduciendonos en este ejemplo podemos definir a una variable objetivo, es decir el elemento de interés que nuestro modelo va a tener estudiar lo mejor posible, como los supervivientes del hundimiento del Titanic('Survived') y a las variables predictoras, es decir las características que ayudarán al modelo a realizar sus predicciones, como el resto del conjunto('Age','Pclass',etc), de esta manera concluimos que se considera un problema de clasficación binaria.

In [4]:
# imputación de datos faltantes mediante medias y modas

df.fillna({"Age":df["Age"].mean()},inplace=True)

mode = df["Embarked"].mode()
df.fillna({"Embarked":mode[0]},inplace=True)

# codificando variables categóricas nominales

df["Sex_encoded"] = LabelEncoder().fit_transform(df["Sex"])

# a su vez generamos una versión textual de la variable binaria objetivo

map_survived = {
    0:"no",
    1:"yes"
}
df["Survived_no_encoded"] = df["Survived"].apply(lambda x : map_survived.get(x))

#### Variable Categóricas y Datasets Desbalanceados
La función de las variables categóricas es brindar información clasificando las observaciones, puenden demostrar una influencia en los datos permitiendonos determinar comportamientos o sacar conclusiones premeditadas si los datos influyen en estas. Los tipos de variables categóricas que hay son nominales: aquellas que actúan como etiquetas de los objetos(dentro de estas pueden encontrarse las que se dividen en 2 clases y su codificación las convierte en binarias, es decir, valores posibles de 0 o 1) y ordinales: aquellas que representan un órden jerárquico en particular que rige en las observaciones.

En algunos casos los Datasets tienden a contener más cantidades de datos categóricos que otros, esto puede llegar a traer problemas la hora de entrenar un modelo o evaluar el desempeño de este debido a que la variable minoritaria tiende a ser ignorada y genera un sesgo en cuanto a la importancia.

In [ ]:
survived_counts = df["Survived_no_encoded"].value_counts().reset_index()
survived_counts.columns = ["survived","count"]

fig = px.bar(survived_counts, x="survived", y="count", title='Conteo de Sobrevivientes')
fig

### CART(classification and regression trees)
algoritmos basados en utilizar umbrales que condicionen el destino de una variable, pero no cualquier umbral, sino aquellos que dependiendo la problemática en cestión(en este caso de clasificación) obtengan el menor valor en cuanto a la probabilidad de clasificar mal la variable objetivo o grado de aleatoriedad del umbral, se realizan repetidas pruebas de este tipo y se determian los umbrales óptimos del modelo. Otra de las características es que se modifica el volumen, es decir, la cantidad de particiones que tendrá el modelo, siendo denominado criterio de parada como el término utilizado. Para ello, hay parámetros que reciben valores y representan cierta particularidad de tamaño como la distancia que habrá entre el nodo(condición) inicial y la última hoja(valor asociado) o la suma mínima de pesos de instancia necesaria en un elemento secundario. Dependiendo de los valores que se les otorguen, el árbol predictor sufrirá de un crecimiento o reducción en cuanto a su complejidad y evitando el sobreajuste en este último caso.

In [ ]:
# seccionamos los datos en grupos de entrenamiento y prueba

x_train, x_test, y_train, y_test = train_test_split(
                                                   df[["Sex_encoded","Pclass","Fare","Parch","SibSp"]],
                                                   df["Survived"],
                                                   test_size=0.12)

# declaración del algoritmo

tree_decision = tree.DecisionTreeClassifier(criterion="entropy", # medida de aleatoriedad, determinará los nodos del modelo en base a la probabilidad de clasificar erroneamente una variable, pero sin darle mayor importancias a ciertas características
                                           max_depth=6, # distancia entre el nodo(condición) principal y la última hoja(respuesta asociada), valores recomendados: 3-10
                                           min_samples_split=6, # mínimo número de muestras requeridas para dividir un nodo, valores recomendados: 2-10
                                           min_samples_leaf=5, #  mínimo número de muestras requeridas en cada hoja, valores recomendados: 1-10
                                           ccp_alpha=0.003) # valor de post-poda(luego de que el modelo se ajute) que evita el sobreajuste 

model = tree_decision.fit(x_train, y_train)

# predicción de los datos de prueba

class_predicts = model.predict(x_test)

class_real = y_test.values

# Matriz de confusión: recolecta los aciertos y desaciertos tanto de la clase postiva como negativa del modelo

matrix_confusion = confusion_matrix(class_real,class_predicts)
TP = matrix_confusion[0,0]
FP = matrix_confusion[0,1]
FN = matrix_confusion[1,0]
TN = matrix_confusion[1,1]

# Accuracy: indica la acertividad general de nuestro modelo con respecto a las nuevas observaciones

accuracy = accuracy_score(class_real, class_predicts)
color_accuracy = "green"
if accuracy < 0.6:
    color_accuracy = "red"
accuracy_str = str(accuracy)

# Recall: es utilizada para poder saber la efectividad de nuestro modelo a la hora de predecir valores de la clase positiva

recall = recall_score(class_real, class_predicts)
color_recall = "green"
if recall < 0.6:
    color_recall = "red"
recall_str = str(recall)

# Precision: es utilizada para poder saber que porcentage de valores que fueron clasificados como positivos son realmente positivos

precision = precision_score(class_real, class_predicts)
color_precision = "green"
if precision < 0.6:
    color_precision = "red"
precision_str = str(precision)

# F1 Score: es utilizada como un resumen de las dos últimas metricas

F1_score = f1_score(class_real, class_predicts)
color_f1 = "green"
if F1_score < 0.6:
    color_f1 = "red"
F1_score_str = str(F1_score)

print(tree.export_text(model, feature_names=["Sex_encoded","Pclass","Fare","Parch","SibSp"]))

plt.figure(figsize=(35,22))
tree.plot_tree(model, feature_names=["Sex_encoded","Pclass","Fare","Parch","SibSp"])
plt.show()

#### Dashboard que muestra las probabilidades de sobrevivir según las variables y el desempeño del modelo

In [ ]:
# creación de dashboard

app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e1_body",children=[
html.H1("Titanic",id="title",className="e1_title"),
html.Div(className="e1_dashboards",children=[
    html.Div(id="graph_div_1",className="e1_graph_div",children=[
        html.Div(id="dropdown_div_1",className="e1_dropdown_div",children=[
            dcc.Dropdown(id="dropdown_1",className="e1_dropdown",
                        options = [
                            {"label":"Sexo","value":"Sex"},
                            {"label":"Clase social","value":"Pclass"},
                            {"label":"Embarcadero","value":"Embarked"},
                            {"label":"Padres e hijos/as","value":"Parch"},
                            {"label":"Hermanas/os y esposas/os","value":"SibSp"},
                        ],
                        value="Sex",
                        multi=False,
                        clearable=False)
        ]),
        dcc.Graph(id="piechart",className="e1_graph",figure={})
    ]),
    html.Div(id="graph_div_2",className="e1_graph_div",children=[
        html.Div(id="dropdown_div_2",className="e1_dropdown_div",children=[
            dcc.Dropdown(id="dropdown_2",className="e1_dropdown",
                        options = [
                            {"label":"Edad","value":"Age"},
                            {"label":"Boleto","value":"Fare"},
                        ],
                        value="Age",
                        multi=False,
                        clearable=False)
        ]),
        dcc.Graph(id="bar",className="e1_graph",figure={})
    ]),
]),
    
    html.Div(className="e1_div", children=[
        html.Div(id="performance", className="e1_performance",children=[
            html.P([html.B("Clases reales", style={"color":"blue"}),"   |   ",html.B("Predicciones",style={"color":"red"})], style={"text-align":"center","font-family":"sans-serif"}),
            html.P("--------------------------------------------------------------------------------------------------------------------------------------",style={"margin":"0"}),
            html.P(f"{class_real}", className="e1_real_class"),
            html.P(f"{class_predicts}", className="e1_predicts")
        ]),
        html.Div(id="metrics", className="e1_metrics", children=[
                html.P("Matriz de confusión", style={"font-size":"0.9em","text-align":"center","font-family":"sans-serif","font-weigth":"bold"}),
                html.Div(id="matrix", className="e1_matrix", children=[
                html.Div([html.B(TP,style={"color":"green","font-family":"sans-serif"})],id="TP",className="e1_successes"), 
                html.Div([html.B(FP,style={"color":"red","font-family":"sans-serif"})],id="FP",className="e1_mistakes"),
                html.Div([html.B(FN,style={"color":"red","font-family":"sans-serif"})],id="FN",className="e1_mistakes"),
                html.Div([html.B(TN,style={"color":"green","font-family":"sans-serif"})],id="TN",className="e1_successes")
                ]),
                html.Div(id="scores",children=[
                html.Ul(id="list",children=[
                html.Li([f"Accuracy: ",html.B(accuracy_str[:4],style={"color":f"{color_accuracy}"})],id="accuracy",className="e1_score"),
                html.Li([f"Recall: ",html.B(recall_str[:4],style={"color":f"{color_recall}"})],id="recall",className="e1_score"),
                html.Li([f"Precision: ",html.B(precision_str[:4],style={"color":f"{color_precision}"})],id="precision",className="e1_score"),
                html.Li([f"F1 Score: ",html.B(F1_score_str[:4],style={"color":f"{color_f1}"})],id="f1_score",className="e1_score")
                ])
                
            ])
        ])
    ])
])

# generan interacción entre los datos de entrada y los elementos que serán modificados 

@app.callback(
    [Output(component_id="piechart",component_property="figure"),
    Output(component_id="bar",component_property="figure")],
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

# función que se ejecuta cada vez que se interactúa con los elementos 

def update_graph(slct_var_cat,slct_var_num):
    
    df_percentage = df.groupby(slct_var_cat)["Survived"].mean().reset_index()
    df_percentage["Survived"] = round(df_percentage["Survived"] * 100)
    df_percentage["Survived"] = df_percentage["Survived"].astype(str)
    df_percentage["var_percentage"] = df_percentage[slct_var_cat].astype(str)+"("+df_percentage["Survived"]+"%)"
    
    piechart = px.pie(df_percentage, values="Survived", names="var_percentage", title="Probabilidad de supervivencia")
    
    df_mean = df.groupby("Survived_no_encoded")[slct_var_num].mean().reset_index()
    
    barplot = px.bar(df_mean, x="Survived_no_encoded", y=slct_var_num, title="Medias de Edad y Boleto",labels={"x":"sobrevivientes","y":slct_var_num})
    barplot.update_layout(xaxis_title="Sobrevivientes")
    
    return piechart,barplot

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de varianza

es la variabilidad que existe en la función objetivo con respecto a los diferentes datos de entrenamiento que se utilicen para la creación del modelo, sucede principalmente en algoritmos que se ajustan fácilmente a los datos y requieren menos suposiciones a diferencia de los algoritmos con alto bías, algunos ejemplos de algoritmos con alta variranza son: KNN, CART o Naive Bayes

In [ ]:
accuracies = []
bucle = list(range(14))

for i in bucle:
    x_train, x_test, y_train, y_test = train_test_split(
                                                   df[["Sex_encoded","Pclass","Fare","Parch","SibSp"]],
                                                   df["Survived"],
                                                   test_size=0.25)
    class_real = y_test.values
    class_predicts = model.predict(x_test)
    accuracy = accuracy_score(class_real, class_predicts)
    accuracies.append(accuracy)
    
plt.plot(bucle, accuracies, marker="*", c="r")
plt.grid("on")
plt.title("Error de Varianza")
plt.ylabel("Acuracies")
plt.show()